In [1]:
## for jupyter notebook only
import warnings

warnings.simplefilter("ignore")

from IPython.display import HTML, display

display(
    HTML(
        data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 95%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""
    )
)


import json
import os

# Importing libraries
import pandas as pd
from ds.utilities.io import ds_trino

pd.options.display.max_columns = None
pd.options.display.max_rows = 50
pd.options.mode.chained_assignment = None


conn = ds_trino.create_trino_connection()

In [2]:
query_mov_7_dias = """

SELECT 

historico.inventory_uuid as ineventory_id
,historico.transaction_uuid as transaction_id 
--,historico.transaction_line_item_index as transaction_line_item_index
,historico.soh_before as soh_before
,historico.soh_after as soh_after
,historico.created_at as historico_created_at


,movimiento.org_uuid as org_uuid,
org.name as org_name,
sku.name, sku.manufacturer_brand_name, sku.sku_id as sku_id
,movimiento.type as type
,movimiento.cause
,movimiento.order_uuid as order_id

,facility.name as facility_name
,facility.country_code
,facility.facility_uuid as facility_uuid
,facility.timezone as facility_timezone
,item.applied_stock_on_hand_delta, item.requested_stock_on_hand_delta,item.stock_on_hand_absolute 
,movimiento.created_at as created_at_UTC
,AT_TIMEZONE(movimiento.created_at, facility.timezone) as created_at_local_time
,cast(AT_TIMEZONE(movimiento.created_at, facility.timezone)as time) as created_at_local_time_only_time
,Date(AT_TIMEZONE(movimiento.created_at, facility.timezone)) as created_at_date




from hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory_history as historico

full join hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory_transaction as movimiento

on historico.transaction_uuid = movimiento.uuid

full join hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory_transaction_line_item as item
on movimiento.uuid = item.inventory_transaction_uuid
left JOIN hudi_ingest.skusku_restaurant_platform_cockroach_dbreplica.sku AS sku
on cast(item.sku_uuid as VARCHAR )= cast(sku.sku_id as VARCHAR )

full join hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory as inventario
on item.inventory_uuid=inventario.uuid
left JOIN hudi_ingest.fos_restaurant_platform_cockroach_dbreplica.facility AS facility 
ON cast(inventario.facility_uuid as VARCHAR ) =cast(facility.facility_uuid as VARCHAR )


left join hudi_ingest.processed.store_config_organization_view as org 
on cast(movimiento.org_uuid as VARCHAR ) =org.organization_id



where facility.country_code in ('MX','CO'--,'BR')
  )
and date_trunc('day',date(movimiento.created_at)) >= DATE_ADD('Day' ,-7, current_date)

and historico.day_partition >='01-01-2022'
and movimiento.day_partition >='01-01-2022'
and item.day_partition >='01-01-2022'
and inventario.day_partition >='01-01-2022'
and sku.day_partition >='2021-01-01'
and facility.day_partition >='2021-01-01'

"""



print("querying ...")
cero_7 = ds_trino.fetch_data(conn=conn, sql_str=query_mov_7_dias, use_cache=False)
print("pulled")


querying ...
pulled


In [3]:
cero_7

,ineventory_id,transaction_id,soh_before,soh_after,historico_created_at,org_uuid,org_name,name,manufacturer_brand_name,sku_id,type,cause,order_id,facility_name,country_code,facility_uuid,facility_timezone,applied_stock_on_hand_delta,requested_stock_on_hand_delta,stock_on_hand_absolute,created_at_UTC,created_at_local_time,created_at_local_time_only_time,created_at_date
0,0e6f379c-b8fa-4145-97e6-bf72a44e2d4c,230f04a3-3a10-497e-8362-c9865abebe3e,2,0,2023-09-11 16:17:42.180,5358f96c-98dc-46f9-a3bb-7500fc708de0,Helados Carmela,"Blueberry, lavanda y miel (500ml)",Helados Carmela,7427580a-60ff-4d05-8d35-ff75ea6345a4,TRANSACTION_MOVE,CAUSE_MANUAL,None,NA-MX-CDMX-Lindavista,MX,da27f06f-01fd-4106-88d9-b3fd41446515,America/Mexico_City,2.0,2.0,NaN,2023-09-11 16:17:42.121,2023-09-11 10:17:42.121 America/Mexico_City,10:17:42.121,2023-09-11
1,f7984a0f-d3ba-4164-b6f8-d7761659397f,230f04a3-3a10-497e-8362-c9865abebe3e,0,2,2023-09-11 16:17:42.180,5358f96c-98dc-46f9-a3bb-7500fc708de0,Helados Carmela,"Blueberry, lavanda y miel (500ml)",Helados Carmela,7427580a-60ff-4d05-8d35-ff75ea6345a4,TRANSACTION_MOVE,CAUSE_MANUAL,None,NA-MX-CDMX-Lindavista,MX,da27f06f-01fd-4106-88d9-b3fd41446515,America/Mexico_City,2.0,2.0,NaN,2023-09-11 16:17:42.121,2023-09-11 10:17:42.121 America/Mexico_City,10:17:42.121,2023-09-11
2,5c5e6a1b-458e-4269-983a-cc28ed5044c1,17142c06-1fd9-4729-90f4-a16516579f58,4,4,2023-09-12 19:39:27.856,5358f96c-98dc-46f9-a3bb-7500fc708de0,Helados Carmela,Bolsa Kraft Chica Helados Carmela,Helados Carmela,6b35e276-6289-4609-95fa-468db52f3765,TRANSACTION_ADJUSTMENT,CAUSE_MANUAL,None,NA-MX-CDMX-Lindavista,MX,da27f06f-01fd-4106-88d9-b3fd41446515,America/Mexico_City,-13.0,-13.0,NaN,2023-09-12 19:39:27.796,2023-09-12 13:39:27.796 America/Mexico_City,13:39:27.796,2023-09-12
3,490ae2be-5032-402d-b6e0-7267d6ec3ac7,17142c06-1fd9-4729-90f4-a16516579f58,1,1,2023-09-12 19:39:27.856,5358f96c-98dc-46f9-a3bb-7500fc708de0,Helados Carmela,Bolsa Kraft Chica Helados Carmela,Helados Carmela,6b35e276-6289-4609-95fa-468db52f3765,TRANSACTION_ADJUSTMENT,CAUSE_MANUAL,None,NA-MX-CDMX-Lindavista,MX,da27f06f-01fd-4106-88d9-b3fd41446515,America/Mexico_City,-13.0,-13.0,NaN,2023-09-12 19:39:27.796,2023-09-12 13:39:27.796 America/Mexico_City,13:39:27.796,2023-09-12
4,8d3eecec-04db-44b1-8b37-f929c13a6d1a,17142c06-1fd9-4729-90f4-a16516579f58,22,27,2023-09-12 19:39:27.856,5358f96c-98dc-46f9-a3bb-7500fc708de0,Helados Carmela,Bolsa Kraft Chica Helados Carmela,Helados Carmela,6b35e276-6289-4609-95fa-468db52f3765,TRANSACTION_ADJUSTMENT,CAUSE_MANUAL,None,NA-MX-CDMX-Lindavista,MX,da27f06f-01fd-4106-88d9-b3fd41446515,America/Mexico_City,-13.0,-13.0,NaN,2023-09-12 19:39:27.796,2023-09-12 13:39:27.796 America/Mexico_City,13:39:27.796,2023-09-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54129,2d95c000-a25d-4645-af0e-4ec19954aa7c,05cd7aac-181e-4144-a7d3-479deae58228,1,1,2023-09-14 17:43:19.322,ae46c14e-d511-440c-97e6-466988eb31d6,Cookievery,Galleta con Conejito,Cookievery,4cb0f8bf-31db-413b-bc09-4ae96f71ae19,TRANSACTION_ADJUSTMENT,CAUSE_MANUAL,None,NA-MX-CDMX-Lindavista,MX,da27f06f-01fd-4106-88d9-b3fd41446515,America/Mexico_City,0.0,0.0,NaN,2023-09-14 17:43:19.260,2023-09-14 11:43:19.260 America/Mexico_City,11:43:19.260,2023-09-14
54130,e6e79c45-37ba-4fb6-82a3-355f85d66d22,05cd7aac-181e-4144-a7d3-479deae58228,1,1,2023-09-14 17:43:19.322,ae46c14e-d511-440c-97e6-466988eb31d6,Cookievery,Galleta con Conejito,Cookievery,4cb0f8bf-31db-413b-bc09-4ae96f71ae19,TRANSACTION_ADJUSTMENT,CAUSE_MANUAL,None,NA-MX-CDMX-Lindavista,MX,da27f06f-01fd-4106-88d9-b3fd41446515,America/Mexico_City,0.0,0.0,NaN,2023-09-14 17:43:19.260,2023-09-14 11:43:19.260 America/Mexico_City,11:43:19.260,2023-09-14
54131,4ec6e0ca-ca7d-4c7e-a18a-f1318141ea7e,05cd7aac-181e-4144-a7d3-479deae58228,1,1,2023-09-14 17:43:19.322,ae46c14e-d511-440c-97e6-466988eb31d6,Cookievery,Galleta con Conejito,Cookievery,4cb0f8bf-31db-413b-bc09-4ae96f71ae19,TRANSACTION_ADJUSTMENT,CAUSE_MANUAL,Non

In [4]:
cero_7.isnull().sum()

ineventory_id                          0
transaction_id                         0
soh_before                             0
soh_after                              0
historico_created_at                   0
org_uuid                               0
org_name                               0
name                                   0
manufacturer_brand_name             1630
sku_id                                 0
type                                   0
cause                                  0
order_id                           45489
facility_name                          0
country_code                           0
facility_uuid                          0
facility_timezone                      0
applied_stock_on_hand_delta          924
requested_stock_on_hand_delta        924
stock_on_hand_absolute             53210
created_at_UTC                         0
created_at_local_time                  0
created_at_local_time_only_time        0
created_at_date                        0
dtype: int64

In [5]:
import numpy as np
import re


In [6]:
#importo secreto
from google.oauth2 import service_account
from ds.utilities.secrets import secrets
import pygsheets
SCOPES = ('https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive')
s = secrets.Secrets()
credentials_sheets = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_latam_service_account"),scopes=SCOPES)
#gc 

In [7]:
## GSheets Oauth
print('Init Auth')
#gc = pygsheets.authorize(custom_credentials=my_credentials)


# open the spreedsheet
print('Opening Spreadsheet')

#ss = gc.open_by_key('1B-NpvDpkufEE4v9ClGFkO9x6AOxWgI1CBuKp7irQJNI')

# select sheet

#ws = ss.worksheet('title','inventario_cero')

print('Clearing Range ...')
#ws.clear(start='A', end='Z')
print('Range CLEARED')

print('Pasting Info ...')
#ws.set_dataframe(cero_7, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [8]:
query_mov_30_dias = """


   


"""



print("querying ...")
#cero_30 = ds_trino.fetch_data(conn=conn, sql_str=query_mov_30_dias, use_cache=False)
print("pulled")


querying ...
pulled


In [9]:
#cero_30

In [10]:
## GSheets Oauth
print('Init Auth')
#gc = pygsheets.authorize(custom_credentials=my_credentials)


# open the spreedsheet
print('Opening Spreadsheet')

#ss = gc.open_by_key('1nBBFPAwj15Z2SBUbr69x3JX_GW7o-tjFAJHTE9Gbtlw')

# select sheet

#ws = ss.worksheet('title','info')

print('Clearing Range ...')
#ws.clear(start='A', end='Z')
print('Range CLEARED')

print('Pasting Info ...')
#ws.set_dataframe(cero_30, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [11]:
query_mov="""
SELECT 

movimiento.uuid as transaction_id,
movimiento.org_uuid,
item.uuid as line_item_id,
item.inventory_uuid as inventory_uuid,

sku.name as sku_name, sku.manufacturer_brand_name, sku.sku_id
,orgs.name as org_name
,movimiento.type
,movimiento.order_uuid as order_id
,movimiento.cause
,facility.name as facility_name
,facility.country_code
,dc.country
,facility.facility_uuid
,item.requested_stock_on_hand_delta,item.applied_stock_on_hand_delta, item.stock_on_hand_absolute, 

movimiento.created_at as created_at_UTC
,date_trunc('day',cast(AT_TIMEZONE(movimiento.created_at, facility.timezone) as DATE  )) as created_day

,cast(AT_TIMEZONE(movimiento.created_at, facility.timezone) as TIMESTAMP ) as created_local_time
--,Date(AT_TIMEZONE(movimiento.created_at, facility.timezone)) as created_at_date
,date_trunc('month',Date(AT_TIMEZONE(movimiento.created_at, facility.timezone))) as created_month
,date_trunc('week', DATE(AT_TIMEZONE(movimiento.created_at, facility.timezone))+ interval '1' day)- interval '1' day as created_week

,movimiento.comment, movimiento.wastage_outcome, movimiento.wastage_reason, movimiento.wastage_reason_notes
,item.adjust_reason, item.adjust_reason_notes

,(
CASE
   WHEN inventario.facility_uuid = 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7' THEN 'LIM - Surquillo'
   WHEN inventario.facility_uuid = '6d2d117b-3fa9-4a2c-85b5-9b86a758f849' THEN 'LIM - Magdalena'
   WHEN inventario.facility_uuid = 'bc2ab3e9-e10d-420a-a3d0-395576fed377' THEN 'LIM - La Molina'
   WHEN inventario.facility_uuid = 'd65136fe-862a-4da5-81ef-580c28f8c61a' THEN 'UIO - 10 de Agosto'
   WHEN inventario.facility_uuid = '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf' THEN 'BOG - Prado'
   WHEN inventario.facility_uuid = '4d78d1e1-5bce-4307-8175-51c816120fce' THEN 'BOG - Castellana'
   WHEN inventario.facility_uuid = 'b2f57126-8e64-4b11-964f-c497a3b53571' THEN 'BOG - Cocinas de la 73'
   WHEN inventario.facility_uuid= 'a5852445-e223-46c9-be99-ba33f204bcd1' THEN 'BOG - Puente Aranda'
   WHEN inventario.facility_uuid = '76a6ef30-ddb2-4178-8c37-86c0d10db4b3' THEN 'BOG - Atabanza'
   WHEN inventario.facility_uuid = 'dd16bf0e-8a89-4f21-bc02-1c9793227502' THEN 'BOG - San Felipe'
   WHEN inventario.facility_uuid = '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82' THEN 'BOG - Cedritos'
   WHEN inventario.facility_uuid = '46fb5e19-3a79-4eda-b119-dee3a331fd81' THEN 'CLO - Chipichape' 
   WHEN inventario.facility_uuid = '58b818c6-615e-4316-b4a3-b9a6bb914c30' THEN 'BAQ - Alto Prado' 
   WHEN inventario.facility_uuid = '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' THEN 'MED - DModa'
   WHEN inventario.facility_uuid= '9eede066-b7e7-43c4-907c-21ece1688b14' THEN 'SJO - Guadalupe' 
   WHEN inventario.facility_uuid = '967f0a2e-9c8e-4b35-929b-37420d9439c9' THEN 'SJO - Pavas'
   WHEN inventario.facility_uuid = '47e2d054-8b82-420c-b1b0-18010f57489c' THEN 'MEX - Insurgentes Sur'
   WHEN inventario.facility_uuid = '565273fe-0074-421d-b1e1-796d1e4390e1' THEN 'MEX - Napoles'
   WHEN inventario.facility_uuid = '864e0a3c-ecba-4305-b3ff-f22edd58c902' THEN 'MEX - Polanco'
   WHEN inventario.facility_uuid = 'bcb04949-3cda-4ca0-be39-b9f579e356ca' THEN 'MEX - Roma'
   WHEN inventario.facility_uuid = '9101d4f9-828f-45c5-b8b6-16986e76b374' THEN 'MEX - Pedregal'
   WHEN inventario.facility_uuid = 'b9545910-bca6-4b2a-a603-363721e7a23d' THEN 'MEX - Santa Fe'
   WHEN inventario.facility_uuid = '394c0103-1c5f-424b-b398-68455fced5bb' THEN 'MTY - Cumbres'
   WHEN inventario.facility_uuid = '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f' THEN 'MTY - Leones NK'
   WHEN inventario.facility_uuid= '988c22e5-20c2-42a5-bc30-989c68ff5eee' THEN 'MTY - Tec'
   WHEN inventario.facility_uuid = '1b90b4ae-6d8f-4a73-bb9e-708c83387430' THEN 'MTY - Santa Maria'
   WHEN inventario.facility_uuid = '88774bba-2e34-4771-8e64-8f059adf02a3' THEN 'MTY - Valle Oriente'
   WHEN inventario.facility_uuid = 'a9a6c4be-32b7-47cf-b783-7a5227a89ed4' THEN 'GDL - Ciudad del Sol'
   WHEN inventario.facility_uuid = 'c66ce711-5486-433f-a9b7-1b36ae3a6dee' THEN 'GDL - Av. Mexico'
   
   WHEN inventario.facility_uuid = '7b5c774f-7739-4d8e-a392-d49547d45fc3' THEN 'BHZ - Contagem Industrial'
   WHEN inventario.facility_uuid = '554263c6-bde4-4d4c-92a7-21ed77e122cd' THEN 'BHZ - Paracatu'
   WHEN inventario.facility_uuid = '522c745a-fa86-42de-83ca-227cbd3cb353' THEN 'BHZ - Arthur Guimaraes' 
   
   WHEN inventario.facility_uuid = '81e6538d-c45b-4479-92e8-fc4e04373e12' THEN 'RIO - Jose Bonifacio'
   WHEN inventario.facility_uuid = '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb' THEN 'RIO - Barra' 
   WHEN inventario.facility_uuid = '976ae838-18e0-40bb-a8dc-360353c01b6d' THEN 'RIO - Niteroi'
   WHEN inventario.facility_uuid = 'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' THEN 'RIO - Tijuca'
  WHEN inventario.facility_uuid = '0ee5dccf-ed5b-465c-9858-b53351353e3b' THEN 'RIO - Camerino'
  WHEN inventario.facility_uuid = '3759ad21-4537-48ea-b1f1-48809e8a03ac' then 'RIO - Botafogo'
   
   WHEN inventario.facility_uuid = '6c601837-403c-425f-a51c-31428ed1a67e' THEN 'SAO - Rua do Acre'
   WHEN inventario.facility_uuid = '87a40094-cf48-4032-8166-e4cee61bce2f' THEN 'SAO - Dino'
   WHEN inventario.facility_uuid = '4326199f-628a-4b00-a6e6-054fb5182c64' THEN 'SAO - Guararapes (Brooklin)'
   WHEN inventario.facility_uuid = 'dcc461ae-c80c-46cc-a5dc-4a5a72761510' THEN 'SAO - Lapa Clelia'
   WHEN inventario.facility_uuid = '7aae3115-311b-449f-88f4-0f1612ecfcee' THEN 'SAO - Borba'
   WHEN inventario.facility_uuid = '88aaaf99-a444-4e0f-99f6-267b635faa53' THEN 'SAO - Vargas'
   when inventario.facility_uuid = '1840180b-9a3b-4fb6-b35c-ef4e6785b26e' THEN 'SAO - Abelardo'
  -- WHEN facility_id= '89cf86b9-9e74-4657-86df-2a851492af88' Then  'SAO - Guarulhos'
   WHEN inventario.facility_uuid = 'a63559d1-93c6-4f60-b683-1e207ccda10a'  THEN 'SAO - Batista'
    WHEN inventario.facility_uuid = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Espanhola'
    when inventario.facility_uuid = '812e6913-d78e-4dbc-877d-16993853aa7e' then 'SAO - Domingos'
    WHEN inventario.facility_uuid = '14d9bb92-6376-4d69-adb6-64f97e4b3561' THEN 'SAO - Ferreira'
    WHEN inventario.facility_uuid = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Espanhola'
  
   
   
   WHEN inventario.facility_uuid = 'da27f06f-01fd-4106-88d9-b3fd41446515' THEN 'MEX - Lindavista'
   
   WHEN inventario.facility_uuid = '4518d99c-fa47-4da7-b5f5-c071c78b5ac2' THEN 'BHZ - Tancredo Neves'
   WHEN inventario.facility_uuid = 'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' THEN 'SCL - Recoleta'
   WHEN inventario.facility_uuid = 'eb13a3b3-88ce-4b59-839a-0c243cea9457' THEN 'SCL - Santiago Centro'
   WHEN inventario.facility_uuid = '4e36823e-98a2-4382-8797-48ef313196a9' THEN 'LIM - Lince'
   WHEN inventario.facility_uuid = 'c559b06b-e193-495c-85cd-a5d1608ca27d' THEN 'BSB - Aguas Claras'
   WHEN inventario.facility_uuid = '035c0131-50e4-465a-9596-ad8eb9be03f8' THEN 'SJO - Heredia'
   WHEN inventario.facility_uuid = '4364cfad-dd86-4232-8164-cc9d51af9ec6' THEN 'SJO - Heredia'
  
  WHEN inventario.facility_uuid = '757182ff-7a54-4571-91d4-0027253c0117' THEN 'BOG - Salitre 2 EM'
   
   

   else facility.name 
   
   
END
) AS facility_sf_name


from 

hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory_transaction as movimiento 
left join hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory_transaction_line_item as item
on cast(movimiento.uuid as VARCHAR ) = item.inventory_transaction_uuid
left JOIN hudi_ingest.skusku_restaurant_platform_cockroach_dbreplica.sku AS sku
on item.sku_uuid = cast(sku.sku_id as VARCHAR )

left join hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory as inventario
on item.inventory_uuid =inventario.uuid
left JOIN hudi_ingest.fos_restaurant_platform_cockroach_dbreplica.facility AS facility 
ON inventario.facility_uuid =cast(facility.facility_uuid as VARCHAR )

--lista de orgs
left join hudi_ingest.processed.store_config_organization_view as orgs
on movimiento.org_uuid =cast(orgs.organization_id as VARCHAR )


JOIN hudi_ingest.scratch.dim_countries dc 
   ON facility.country_code = dc.country_code AND region in ('LATAM','USC'
                                                                            )




where movimiento.day_partition >='2000-01-01'
and item.day_partition >='2000-01-01'
and inventario.day_partition>='2000-01-01'
and sku.day_partition>='2000-01-01'
and facility.day_partition>='2000-01-01'
And facility.country_code in ('MX','CO'--,'BR'--,'US'--,'EC','PE','PR','CL','GT','CR','DO','SV'
)
and date_trunc('day',date(movimiento.created_at)) >= DATE_ADD('Day' ,-7, current_date)



"""

print("querying ...")
mov= ds_trino.fetch_data(conn=conn, sql_str=query_mov, use_cache=False)
print("pulled")


querying ...
pulled


In [12]:
mov

,transaction_id,org_uuid,line_item_id,inventory_uuid,sku_name,manufacturer_brand_name,sku_id,org_name,type,order_id,cause,facility_name,country_code,country,facility_uuid,requested_stock_on_hand_delta,applied_stock_on_hand_delta,stock_on_hand_absolute,created_at_UTC,created_day,created_local_time,created_month,created_week,comment,wastage_outcome,wastage_reason,wastage_reason_notes,adjust_reason,adjust_reason_notes,facility_sf_name
0,b1979610-e31b-4ffe-9d40-ac27cae14382,b556b267-8821-4162-a0cc-254fc70dafa0,8acb6cf3-3309-4c58-8189-07460ca337a6,1fce0d03-db67-43f0-b12b-bfeb2192ee90,Bizcocho Inglés,Tere Cazola,97baefab-7c5e-4ebf-b321-a5353ac66448,Tere Cazola,TRANSACTION_SOLD,e150aa2e-e7ed-3b71-a55d-da1ca9a88fd9,CAUSE_ORDER,NA-MX-CDMX-Lindavista,MX,Mexico,da27f06f-01fd-4106-88d9-b3fd41446515,-1.0,-1.0,NaN,2023-09-15 04:59:47.985,2023-09-14,2023-09-14 22:59:47.985,2023-09-01,2023-09-10,,None,None,None,ADJUST_REASON_UNKNOWN,,MEX - Lindavista
1,47b4b149-4775-4601-8576-aa0cb3f1b04f,b556b267-8821-4162-a0cc-254fc70dafa0,54519026-a1b4-40a6-acd4-dbd41e79c060,3d4df99e-e061-4ac8-9509-26fcdeeee1c8,Rebanada Capuchino,Tere Cazola,67f5bf55-d2f0-4c97-b0cd-82bc74e64f5f,Tere Cazola,TRANSACTION_ADJUSTMENT,None,CAUSE_MANUAL,NA-MX-CDMX-Lindavista,MX,Mexico,da27f06f-01fd-4106-88d9-b3fd41446515,NaN,NaN,0.0,2023-09-15 13:53:14.036,2023-09-15,2023-09-15 07:53:14.036,2023-09-01,2023-09-10,Created through CSS Adjustment API.,None,None,None,ADJUST_REASON_UNKNOWN,,MEX - Lindavista
2,b70c09bc-edbc-4eac-9637-31d6a3111fe5,b556b267-8821-4162-a0cc-254fc70dafa0,9f470507-8d69-4f84-ad5e-607154aa449f,0b17405b-b880-45f4-9422-7d9f2df7d591,Tres Leches Chantilly,Tere Cazola,6174c830-a363-4b2e-9dcf-0624cd6c3417,Tere Cazola,TRANSACTION_SOLD,fc7c34bb-8dae-3911-915a-e3d7c259ad75,CAUSE_ORDER,NA-MX-CDMX-Lindavista,MX,Mexico,da27f06f-01fd-4106-88d9-b3fd41446515,-1.0,-1.0,NaN,2023-09-15 18:48:06.980,2023-09-15,2023-09-15 12:48:06.980,2023-09-01,2023-09-10,,None,None,None,ADJUST_REASON_UNKNOWN,,MEX - Lindavista
3,40ffe9d7-4dc0-4548-8592-b2838224db53,b556b267-8821-4162-a0cc-254fc70dafa0,5a7ad220-6c05-4626-92b7-3d820f7935dd,97becd7f-fdda-41a4-9bec-72e48445828f,Cheese Cake Doble Chocolate Rebanada,Tere Cazola,7f789f3d-7aaf-4dc7-b95d-6ff5c80aa27f,Tere Cazola,TRANSACTION_SOLD,1d8660f9-e02e-3fcf-8ec6-9116bdd6bb1b,CAUSE_ORDER,NA-MX-CDMX-Lindavista,MX,Mexico,da27f06f-01fd-4106-88d9-b3fd41446515,-1.0,-1.0,NaN,2023-09-16 03:48:54.432,2023-09-15,2023-09-15 21:48:54.432,2023-09-01,2023-09-10,,None,None,None,ADJUST_REASON_UNKNOWN,,MEX - Lindavista
4,40ffe9d7-4dc0-4548-8592-b2838224db53,b556b267-8821-4162-a0cc-254fc70dafa0,c5de8662-59b1-400f-924f-d267edfaaecf,ff76657c-cf82-4bc0-8104-b94672623918,Cheese Cake Doble Chocolate Rebanada,Tere Cazola,7f789f3d-7aaf-4dc7-b95d-6ff5c80aa27f,Tere Cazola,TRANSACTION_SOLD,1d8660f9-e02e-3fcf-8ec6-9116bdd6bb1b,CAUSE_ORDER,NA-MX-CDMX-Lindavista,MX,Mexico,da27f06f-01fd-4106-88d9-b3fd41446515,-1.0,-1.0,NaN,2023-09-16 03:48:54.432,2023-09-15,2023-09-15 21:48:54.432,2023-09-01,2023-09-10,,None,None,None,ADJUST_REASON_UNKNOWN,,MEX - Lindavista
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33131,10369b8e-b7be-4a0f-a1f7-899ccfa1d64c,b556b267-8821-4162-a0cc-254fc70dafa0,7fcecfba-0aad-40ad-8e04-629832fd20e9,fb3848bb-0bc1-4bf8-87ad-282509663dac,Rebanada Tres Leches Extreme Queso de Bola,Tere Cazola,880682c8-0d2b-4677-a652-e9b3a82b161b,Tere Cazola,TRANSACTION_ORDER_SCANOUT,957d7cda-8d7f-31a7-9a11-d89950e78c84,CAUSE_MANUAL,NA-MX-CDMX-Roma*,MX,Mexico,bcb04949-3cda-4ca0-be39-b9f579e356ca,NaN,NaN,0.0,2023-09-12 22:25:53.395,2023-09-12,2023-09-12 16:25:53.395,2023-09-01,2023-09-10,Order zeroing adjustment from SOURCE_STOCK_UNK...,None,None,None,ADJUST_REASON_WASTE_EXPIRED,,MEX - Roma
33132,a1da6c54-e334-4b94-b034-1193a5770a7c,b556b267-8821-4162-a0cc-254fc70dafa0,664e08d2-babf-461c-bd1b-64e2e5b35c27,fb3848bb-0bc1-4bf8-87ad-282509663dac,Rebanada Tres Leches Extreme Queso de Bola,Tere Cazola,880682c8-0d

In [13]:
## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1PS10CWP6Jbkt83IqzAoxtAcxEMTgoSIH8abQC83PEZ0')

# select sheet

ws = ss.worksheet('title','Query transacciones inventario')

print('Clearing Range ...')
ws.clear(start='A', end='Z')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(mov, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!
